In [37]:
%matplotlib inline
import obspy
import glob
import os
import librosa
import numpy as np
import scipy as sp
import pandas as pd
import visuals as vs
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, GridSearchCV

random_state = 6
np.random.seed(random_state)

In [38]:
def read_seismogram(filename):
    st = obspy.read(filename)
    return st

def extract_features(seismogram, signal_label):
    data = seismogram[0].data
    sample_rate = seismogram[0].stats.sampling_rate
    stft = np.abs(librosa.stft(data))
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S = stft, sr = sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(data, sr = sample_rate).T,axis=0)
    moment = sp.stats.moment(data)
    variation = sp.stats.variation(data)
    skew = sp.stats.skew(data)
    var = np.var(data)
    autocr = np.correlate(data, data)
    kurto = sp.stats.kurtosis(data)
    return np.hstack([mfccs,chroma,mel,moment,variation, skew, var, autocr, kurto, signal_label])

def parse_and_stack_seismograms(parent_dir, sub_dirs):
    
    features = np.empty((0,187))
    if parent_dir == 'seismograms/explosions/':
        signal_label = 1
    elif parent_dir == 'seismograms/earthquakes/':
        signal_label = 0
        
    for indx, sub_dir in enumerate(sub_dirs):
        for filename in glob.glob(os.path.join(parent_dir, sub_dir, '*.SAC')):
            seismogram = read_seismogram(filename)
            single_feature= extract_features(seismogram, signal_label)
            features = np.vstack([features, single_feature])
        
    dataFrame = pd.DataFrame(features)
    return dataFrame

In [39]:
parent_dir = 'seismograms/explosions/'
sub_dirs = ['1998-05-11-mb52-india','1998-05-28-mb48-pakistan', '1998-05-30-mb46-pakistan', '2013-02-12-mb51-north-korea', '2016-01-06-mb51-north-korea', '2017-09-03-mb63-north-korea']
df_explosions  = parse_and_stack_seismograms(parent_dir,sub_dirs)

/usr/local/lib/python3.6/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [40]:
parent_dir = 'seismograms/earthquakes/'
sub_dirs = ['2004-12-26-mw90-sumatra', '2010-03-12-mw55-myanmar-india-border-region', '2017-08-15-mb49-southeast-of-ryukyu-islands', '2017-09-08-mww81-near-coast-of-chiapas-mexico']
df_earthquakes  = parse_and_stack_seismograms(parent_dir,sub_dirs)

/usr/local/lib/python3.6/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
/usr/local/lib/python3.6/site-packages/obspy/core/trace.py:187: UserWarning: Calibration factor set to 0.0!
  warnings.warn(msg, UserWarning)


In [60]:
frames = [df_explosions, df_earthquakes]
df = pd.concat(frames)
X = df.iloc[:,:186]
Y = df.iloc[:,-1]

(1606, 187)
(1606, 186)


In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=random_state)
mms = StandardScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.fit_transform(X_test)

In [64]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
rfc = RandomForestClassifier(criterion='entropy', random_state = random_state, class_weight='balanced')
params = {
    'n_estimators': [10, 20, 50, 100],
    'max_depth': [1, 2, 3, 4, 5],
    'min_samples_split': [10, 20, 40, 50]
}

grid_clf = GridSearchCV(estimator = rfc, param_grid = params, scoring = 'accuracy', cv = cv)

In [65]:
# we have to find out best optimized parameters
grid_clf.fit(X_train, Y_train.astype(int))

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=6, shuffle=True),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=6, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 20, 50, 100], 'max_depth': [1, 2, 3, 4, 5], 'min_samples_split': [10, 20, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [66]:
# get the best model
print('\n # Best estimator ---------\n{}'.format(grid_clf.best_estimator_))

# Get the best parameters
print('\n # Best parameters ---------\n{}'.format(grid_clf.best_params_))

# best score
print('\n # Best score ---------\n{}'.format(grid_clf.best_score_))


 # Best estimator ---------
RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=6, verbose=0, warm_start=False)

 # Best parameters ---------
{'max_depth': 5, 'min_samples_split': 10, 'n_estimators': 100}

 # Best score ---------
0.8211743772241993


In [70]:
Y_pred = grid_clf.predict(X_test)

In [71]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print('The accuracy of the model is {}'.format(accuracy_score(Y_test, Y_pred)))
print('# Classification report \n {}'.format(classification_report(Y_test, Y_pred)))

The accuracy of the model is 0.6887966804979253
# Classification report 
              precision    recall  f1-score   support

        0.0       0.69      0.99      0.81       322
        1.0       0.78      0.09      0.16       160

avg / total       0.72      0.69      0.59       482



In [69]:
confusion_matrix(Y_pred, Y_test)

array([[318, 146],
       [  4,  14]])

In [73]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X, Y)
Y_pred = clf.predict(X_test)

In [74]:
print('The accuracy of the model is {}'.format(accuracy_score(Y_test, Y_pred)))
print('# Classification report \n {}'.format(classification_report(Y_test, Y_pred)))

The accuracy of the model is 0.6680497925311203
# Classification report 
              precision    recall  f1-score   support

        0.0       0.67      1.00      0.80       322
        1.0       0.00      0.00      0.00       160

avg / total       0.45      0.67      0.54       482



/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
